<a href="https://colab.research.google.com/github/eric15342335/realfill/blob/main/get_all_best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/eric15342335/realfill
%cd realfill

In [ ]:
!curl -L https://github.com/eric15342335/realfill/releases/download/dataset/realfill_data_release_full.zip -o realfill_data_release_full.zip
!unzip -q realfill_data_release_full.zip

In [ ]:
%pwd
%ls -lh

In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

In [ ]:
import os
from pathlib import Path
import subprocess
import re

# --- Configuration Toggles ---
USE_DRIVE_STORAGE = True       # True: Use Google Drive; False: Use local Colab storage

# --- Base Environment Setup ---
DRIVE_BASE_DIR = "/content/drive/MyDrive/RealFill" # Base path on Google Drive


def run_loftr_and_update_best():
    """
    For all folders in the current directory containing "result" in their name,
    runs loftr_ranking.py, gets the highest ranking image, and updates
    "best.txt" in the current directory.
    """
    current_dir = DRIVE_BASE_DIR
    best_txt_path = os.path.join(current_dir, "best.txt")
    results = {}

    # Find folders containing "result"
    result_folders = [
        f for f in os.listdir(current_dir)
        if os.path.isdir(os.path.join(current_dir, f)) and "result" in f
    ]

    if not result_folders:
        print("No folders containing 'result' found in the current directory.")
        return

    # Process each folder
    for folder_name in result_folders:
        folder_path = os.path.join(current_dir, folder_name)
        script_path = os.path.join(current_dir, "tools", "loftr_ranking.py")  # Corrected path

        if not os.path.exists(script_path):
            print(f"Error: loftr_ranking.py not found at {script_path}")
            return

        try:
            # Run loftr_ranking.py
            command = ['python', script_path, folder_path]
            process = subprocess.run(command, check=True, capture_output=True, text=True)
            output_text = process.stdout

            # Extract highest ranking image
            match = re.search(r"Highest ranking image: (.*)", output_text)
            if match:
                highest_ranking_image_name = match.group(1).strip()
                results[folder_name] = highest_ranking_image_name
            else:
                print(f"Warning: No highest ranking image found in output for folder {folder_name}")
                results[folder_name] = "No image found"

        except subprocess.CalledProcessError as e:
            print(f"Error running loftr_ranking.py on {folder_name}: {e}")
            print(f"Command: {e.cmd}")
            print(f"Output: {e.output}")
            results[folder_name] = "Error running script"
        except FileNotFoundError:
            print("Error: python command not found.  Ensure Python is installed and in your system's PATH.")
            results[folder_name] = "Python not found"

    # Update best.txt
    with open(best_txt_path, "w") as f:
        f.write("Results:\n")
        for folder, image_name in results.items():
            f.write(f"{folder}: {image_name}\n")
    print(f"Results written to {best_txt_path}")



if __name__ == "__main__":
    run_loftr_and_update_best()
    print("Done")
